In [ ]:
# General notebook settings
import warnings

warnings.filterwarnings("error", category=DeprecationWarning)


# Quickstart 3 - Investments & Storage


## Problem Description

A data centre in Seville, Spain, has a constant demand of 100 MW. The operator
considers investing in on-site solar PV and battery storage to reduce reliance
on grid electricity, which is priced at 120 €/MWh. The investment costs and
characteristics of the components are as follows:


| Component | Overnight Cost | Lifetime | Discount Rate |
|-----------|----------------|----------|---------------|
| Solar PV  | 400 €/kW       | 25 years | 5%            |
| Battery Storage   | 150 €/kWh      | 25 years | 5%            |
| Battery Inverter  | 170 €/kW       | 10 years | 5%            |

- The battery storage system has a round-trip efficiency of 90% and an energy-to-power ratio of 4 hours.
- The solar PV plant has a capacity factor time series given [here](https://model.energy/data/time-series-f17c3736a2719ce7da58484180d89e2d.csv).
- Assume that feeding electricity into the grid is not allowed.

Find the least-cost investment in solar PV and battery storage to cover the load. What is the average cost per unit of electricity consumed?
How much electricity is consumed from the grid and when? How is the battery operated?

## PyPSA Solution

We start by creating a new network with a single bus and the data centre load.
We also add a generator to model supply from the grid priced at 120 €/MWh.
These are the fixed components of the network.

In [ ]:
import numpy as np
import pandas as pd

import pypsa
from pypsa.common import annuity

n = pypsa.Network()

n.add("Bus", "seville")

n.add("Load", "demand", bus="seville", p_set=100)

n.add("Generator", "grid", bus="seville", p_nom=100, marginal_cost=120, carrier="grid");

Next, we read in the capacity factor time series of the network, which covers hourly data for the year 2011.

In [ ]:
p_max_pu = pd.read_csv(
    "https://model.energy/data/time-series-f17c3736a2719ce7da58484180d89e2d.csv",
    index_col=0,
    parse_dates=True,
)["solar"]
p_max_pu[7:15]

We need to tell PyPSA that these are the snapshots (time steps) we want to optimise over.

In [ ]:
n.set_snapshots(p_max_pu.index)
len(n.snapshots)

Then, we add the solar PV with the availability time series as `p_max_pu`, the annualised costs in €/MW/a as `capital_cost` and mark the component as extendable with `p_nom_extendable`.

In [ ]:
n.add(
    "Generator",
    "solar",
    bus="seville",
    p_max_pu=p_max_pu,
    capital_cost=annuity(0.05, 25) * 400_000,
    p_nom_extendable=True,
    carrier="solar",
);

Similarly, we add the battery storage. Here, we need to take extra care with the multiple cost components of the battery system for the `capital_cost`, and the energy-to-power ratio (`max_hours`).

In [ ]:
cc_inverter = annuity(0.05, 25) * 170_000
cc_storage = annuity(0.05, 25) * 150_000

n.add(
    "StorageUnit",
    "battery",
    bus="seville",
    capital_cost=cc_inverter + 4 * cc_storage,
    p_nom_extendable=True,
    carrier="battery",
    efficiency_store=np.sqrt(0.9),
    efficiency_dispatch=np.sqrt(0.9),
    max_hours=4,
);

Now, the model is ready to be solved:

In [ ]:
n.optimize(log_to_console=False)

To retrieve the optimised capacities, we can either directly access the `p_nom_opt` attribute of the components, or use the `n.statistics` module.

In [ ]:
display(n.generators.p_nom_opt)
display(n.storage_units.p_nom_opt)

In [ ]:
n.statistics.optimal_capacity()

The statistics module also provides a convenient way to calculate investment and operational costs, as well as the average cost per unit of electricity consumed.

In [ ]:
totex = {"opex": n.statistics.opex(), "capex": n.statistics.capex()}
pd.concat(totex, axis=1).div(1e6).round(2)  # M€/a

In [ ]:
(n.statistics.capex().sum() + n.statistics.opex().sum()) / 100 / 8760  # €/MWh

The statistics module can also give you the energy balances of the system, to see how much electricity is consumed from the grid and when, and what the battery storage losses are.

In [ ]:
n.statistics.energy_balance().div(1e3)  # GWh

To access and plot the state of charge profile of the battery for January, run

In [ ]:
n.storage_units_t.state_of_charge.loc["2011-01"].plot(backend="plotly")

The statistics functions also have built-in plotting capabilities, e.g. to plot the dispatch profiles of the system as stacked area charts.

In [ ]:
n.add(
    "Carrier",
    ["grid", "solar", "battery", "AC"],
    color=["blue", "yellow", "green", "k"],
)

n.statistics.energy_balance.iplot()

Finally, any network object can be exported to files, for example to Excel or NetCDF, for further analysis or reporting.  Importing from files is of course also possible.

In [ ]:
n.export_to_excel("data-centre-investment.xlsx")
n.export_to_netcdf("data-centre-investment.nc")

o = pypsa.Network("data-centre-investment.nc")

Find many more extensive examples in the [examples](examples.md) section.

The [user guide](user-guide.md) section contains detailed information on architecture, components, problem formulation and utilities.